In [2]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics

unable to import 'smart_open.gcs', disabling that module


# Divide train into 90 % training and 10 % testing

In [2]:
all_features = ["text_ tokens", "tweet_id" ,"language", "tweet_timestamp", "engaged_with_user_id",\
               "engaging_user_id", "engagee_follows_engager", "retweet_timestamp"]
train = pd.read_csv(open("s3://recsys-challenge-2020/training.tsv"), encoding="utf-8",
                    names = all_features, usecols= [0, 2, 7, 8, 9, 14, 19, 21], sep="\x01"
                   )
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))
val_set['retweet_bool'] = val_set.retweet_timestamp.fillna(0)
val_set[val_set.retweet_bool != 0.0] = 1.0
gt = list(val_set.retweet_bool)
pr = [0.5] * len(gt)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [ ]:
print(prauc)
print(rce)

### Metrics on FM

In [3]:
all_features = ["tweet_timestamp", "retweet_timestamp"]
train = pd.read_csv(open("s3://recsys-challenge-2020/training.tsv"), encoding="utf-8",
                    names = all_features, usecols= [8, 21], sep="\x01"
                   )
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()
val_set['retweet_bool'] = val_set.retweet_timestamp.fillna(0)
val_set.loc[val_set.retweet_bool != 0.0, 'retweet_bool'] = 1.0
gt = list(val_set.retweet_bool)
pr_df = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/data/retweet/val_set_result.csv',\
                      names = ['probability_retweet'])
pr = list(pr_df.probability_retweet)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [4]:
prauc

0.42136331079050626

In [5]:
rce

18.46998582987537

In [3]:
all_features = ["tweet_timestamp", "retweet_timestamp"]
train = pd.read_csv(open("s3://recsys-challenge-2020/training.tsv"), encoding="utf-8",
                    names = all_features, usecols= [8, 21], sep="\x01"
                   )
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()
val_set['retweet_bool'] = val_set.retweet_timestamp.fillna(0)
val_set.loc[val_set.retweet_bool != 0.0, 'retweet_bool'] = 1.0
gt = list(val_set.retweet_bool)
pr_df = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/retweet/val_set.output',\
                      names = ['probability_retweet'])
pr = list(pr_df.probability_retweet)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [4]:
prauc

0.4696163566300273

In [5]:
rce

23.62294508164885

In [10]:
gt_df = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/retweet/val_set_gt.csv',\
            names = ['ground_truth'])
gt = list(gt_df.ground_truth)
pr_df = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/ffm/retweet/val_set.output',\
                      names = ['probability_retweet'])
pr = list(pr_df.probability_retweet)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

In [11]:
prauc

0.47210513831601136

In [12]:
rce

23.841255320153753

In [3]:
gt_df = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/ft_embedding_based_features/ffm/retweet/val_set_gt.csv',\
            names = ['ground_truth'])
gt = list(gt_df.ground_truth)
pr_df = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/ft_embedding_based_features/ffm/retweet/val_set.output',\
                      names = ['probability_retweet'])
pr = list(pr_df.probability_retweet)
prauc = metrics.compute_prauc(pr,gt)
rce = metrics.compute_rce(pr,gt)

##### ./ffm-train -p ../../ft_embedding_based_features/ffm/retweet/val_set.ffm -l 0.0002 --auto-stop -s 96 ../../ft_embedding_based_features/ffm/retweet/train_set.ffm
##### ./ffm-predict ../../ft_embedding_based_features/ffm/retweet/val_set.ffm train_set.ffm.model ../../ft_embedding_based_features/ffm/retweet/val_set.output
##### iter   tr_logloss   va_logloss      tr_time
#####   1      0.29747      0.25389       1129.7
#####   2      0.24647      0.24008       2212.0
#####   3      0.21755      0.23859       3215.6
#####   4      0.19949      0.24003
##### Auto-stop. Use model at 3th iteration.

In [4]:
prauc

0.49112814598621485

In [5]:
rce

25.663740331606256